# FLASK Web-App

Visualize and Deployment on Flask Web-App

#### Notes:
1. Flask-App for Data Visualization
2. Crawling and Pre-Processing / Target / Sentiment-Identification done separately
3. Sentiment-Results will be appended on a result-csv, and charts will be created using python plotly library
4. Topic Modelling would be displayed using python pyLDAvis library, and saved in html templates
5. Templates will be loaded on Flask using HTML/Javascript

### HTML Template (BootStrap):
Reference: https://bootstrapmade.com/iportfolio-bootstrap-portfolio-websites-template/

### Sticky Navigation Bar:
Reference: https://www.w3schools.com/howto/howto_js_navbar_sticky.asp

### Datepicker JS Example Flask: 

Reference FLASK-HTML Implementation: https://gist.github.com/doobeh/3e685ef25fac7d03ded7

Reference (Date Parsing): https://stackoverflow.com/questions/52682074/parsing-dates-using-wtforms-flask-python



### Serve Static Files in Flask
Reference: https://stackoverflow.com/questions/20646822/how-to-serve-static-files-in-flask
- (Pre-Saved) HTML files for LDA

### Plot graph objects using Plotly

- Reference (Line graphs): https://plotly.com/python/line-charts/ 
- Reference (Pie Chart): https://plotly.github.io/plotly.py-docs/generated/plotly.graph_objects.Pie.html
- JSON chart Schema: https://plotly.com/chart-studio-help/json-chart-schema/
- Plotly Python Figure Reference: https://plotly.com/python/reference/ 

- Color RGB: https://www.rapidtables.com/web/color/red-color.html

### Session Data (session, g)
- Load data on initialization
- Reference: https://pythonise.com/series/learning-flask/python-before-after-request
- Explanation between g and session: https://stackoverflow.com/questions/32909851/flask-session-vs-g

In [1]:
from flask import Flask, render_template, url_for, request, session, g
from flask_wtf import Form
from wtforms import DateField
from datetime import date

import pandas as pd
import numpy as np
import json 
import plotly
import plotly.graph_objs as go

In [2]:
# Pie Chart for Sentiment
def create_SentimentPlot(df, date, target='trump', channel='twit'):
        
    data = [
        go.Pie(
            labels=['Positive', 'Negative', 'Neutral'],
            values=df.loc[date,[channel+'_'+target+'_pos', channel+'_'+target+'_neg', channel+'_'+target+'_neu']].to_list(),
            marker=dict(colors=['#98FB98', '#F08080', '#C0C0C0']),
            name=target
        ),
    ]
    graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON



# Line Chart for Overall Mentions 
def create_MentionPlot(df):
    
    data = [
        # Trump: Total-Mentions, Twitter-Mentions, Reddit-Mentions
        go.Scatter(x=df.index, y=df['twit_trump'] + df['redd_trump'],mode='lines',name='Trump Mentions (Total)'),
        go.Scatter(x=df.index,y=df['twit_trump'],mode='lines',name='Trump Mentions (Twitter)'),
        go.Scatter(x=df.index,y=df['redd_trump'],mode='lines',name='Trump Mentions (Reddit)'),
        # Biden: Total-Mentions, Twitter-Mentions, Reddit-Mentions
        go.Scatter(x=df.index,y=df['twit_biden'] + df['redd_biden'],mode='lines',name='Biden Mentions (Total)'),
        go.Scatter(x=df.index,y=df['twit_biden'],mode='lines',name='Biden Mentions (Twitter)'),
        go.Scatter(x=df.index,y=df['redd_biden'],mode='lines',name='Biden Mentions (Reddit)')
    ]
    graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON



# Line Chart for Overal Stance Prediction (TextBlob)
# Assume (Pos-For + Neg-Against)
def create_StancePlot_Vader(df):
    
    # Weightage
    # Twitter-Weightage = Twitter-Data / ( Twitter-Data + Reddit Data )
    # Reddit-Weightage = Reddit-Data / ( Twitter-Data + Reddit Data )
    TwitWeight = df['twit_all'] / ( df['twit_all'] + df['redd_all'] )
    ReddWeight = df['redd_all'] / ( df['twit_all'] + df['redd_all'] )

    # Trump-Aggregation
    # Trump-Twitter-Score = ( Twit_TrumpPos + Twit_BidenNeg ) / Twitter-Data
    # Trump-Reddit-Score = ( Twit_TrumpPos + Twit_BidenNeg ) / Reddit-Data
    # Trump-Stance = Twitter-Score * Twitter-Weightage + Reddit-Score * Reddit-Weightage
    TwitTrump = ( df['twit_trump_pos'] + df['twit_biden_neg'] ) / df['twit_all']
    ReddTrump = ( df['redd_trump_pos'] + df['redd_biden_neg'] ) / df['redd_all']
    TrumpStance = ( TwitTrump * TwitWeight ) + ( ReddTrump * ReddWeight )

    # Biden-Aggregation
    # Biden-Twitter-Score = ( Twit_BidenPos + Twit_TrumpNeg ) / Twitter-Data
    # Biden-Reddit-Score = ( Twit_BidenPos + Twit_TrumpNeg ) / Reddit-Data
    # Biden-Stance = Twitter-Score * Twitter-Weightage + Reddit-Score * Reddit-Weightage
    TwitBiden = ( df['twit_trump_neg'] + df['twit_biden_pos'] ) / df['twit_all']
    ReddBiden = ( df['redd_trump_neg'] + df['redd_biden_pos'] ) / df['redd_all']
    BidenStance = ( TwitBiden * TwitWeight ) + ( ReddBiden * ReddWeight )

    data = [
        # Trump-Stance (Aggregation): 
        go.Scatter(
            y=TrumpStance, #df['twit_trump_pos'] / df['twit_all_pos'],
            x=df.index,mode='lines',name='Trump Sentiment'
        ),
        # Biden: Twitter-Biden-Pos / Twit-Pos-All
        go.Scatter(
            y=BidenStance, #df['twit_biden_pos'] / df['twit_all_pos'],
            x=df.index,mode='lines',name='Biden Sentiment'
        )
    ]
    graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON



# Line Chart for Overal Stance Prediction (TextBlob)
# Assume (Pos-For + Neg-Against)
def create_StancePlot_TextBlob(df):
    
    # Weightage
    # Twitter-Weightage = Twitter-Data / ( Twitter-Data + Reddit Data )
    # Reddit-Weightage = Reddit-Data / ( Twitter-Data + Reddit Data )
    TwitWeight = df['tb_twit_all'] / ( df['tb_twit_all'] + df['tb_redd_all'] )
    ReddWeight = df['tb_redd_all'] / ( df['tb_twit_all'] + df['tb_redd_all'] )

    # Trump-Aggregation
    # Trump-Twitter-Score = ( Twit_TrumpPos + Twit_BidenNeg ) / Twitter-Data
    # Trump-Reddit-Score = ( Twit_TrumpPos + Twit_BidenNeg ) / Reddit-Data
    # Trump-Stance = Twitter-Score * Twitter-Weightage + Reddit-Score * Reddit-Weightage
    TwitTrump = ( df['tb_twit_trump_pos'] + df['tb_twit_biden_neg'] ) / df['tb_twit_all']
    ReddTrump = ( df['tb_redd_trump_pos'] + df['tb_redd_biden_neg'] ) / df['tb_redd_all']
    TrumpStance = ( TwitTrump * TwitWeight ) + ( ReddTrump * ReddWeight )

    # Biden-Aggregation
    # Biden-Twitter-Score = ( Twit_BidenPos + Twit_TrumpNeg ) / Twitter-Data
    # Biden-Reddit-Score = ( Twit_BidenPos + Twit_TrumpNeg ) / Reddit-Data
    # Biden-Stance = Twitter-Score * Twitter-Weightage + Reddit-Score * Reddit-Weightage
    TwitBiden = ( df['tb_twit_trump_neg'] + df['tb_twit_biden_pos'] ) / df['tb_twit_all']
    ReddBiden = ( df['tb_redd_trump_neg'] + df['tb_redd_biden_pos'] ) / df['tb_redd_all']
    BidenStance = ( TwitBiden * TwitWeight ) + ( ReddBiden * ReddWeight )

    data = [
        # Trump-Stance (Aggregation): 
        go.Scatter(
            y=TrumpStance, #df['twit_trump_pos'] / df['twit_all_pos'],
            x=df.index,mode='lines',name='Trump Sentiment'
        ),
        # Biden: Twitter-Biden-Pos / Twit-Pos-All
        go.Scatter(
            y=BidenStance, #df['twit_biden_pos'] / df['twit_all_pos'],
            x=df.index,mode='lines',name='Biden Sentiment'
        )
    ]
    graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON



In [4]:
app = Flask(__name__)
# By default static html files will be saved under directory "static"
app.secret_key = 'SHH!'




class DateForm(Form):
    dt = DateField('Pick a Date', format="%m/%d/%Y")

@app.before_first_request
def before_first_request_func():
    #print("This function will run once")
    # use session to store data across specific browser/session
    # use g to store data across specific request lifecycle
    session['N']=40
    
    
## Home Tab

@app.route('/', methods=['post','get'])
def myIndex():
    
    # Create Userform instances, for Date-Picker
    form = DateForm() 
    
    # Load Data
    df = pd.read_csv('static/data.csv')
    df.set_index('date', inplace=True)

    
    # Get Date (from userform)
    if request.method == "POST" and form.validate():
        varDate = form.dt.data.strftime('%m/%d/%Y')
        message = "this is a message1"
    else:
        # Default: use last row in DataFrame if nothing selected
        varDate = df.index[-1]
        message = 'this is a default date'
    
    
    # 1. Create Plot (line): Mentions (for each candidate) over Time
    plotMention = create_MentionPlot(df=df)
    # 2. Create Plot (line): Stance Prediction (for each candidate) over Time
    plotStanceVader = create_StancePlot_Vader(df=df)
    plotStanceTB = create_StancePlot_TextBlob(df=df)
    # 3.1. Create Plot (bar): Sentiment Analysis for Trump
    plotTrumpTwitVD = create_SentimentPlot(df=df, date=varDate, target='trump', channel='twit')
    plotTrumpReddVD = create_SentimentPlot(df=df, date=varDate, target='trump', channel='redd')
    plotTrumpTwitTB = create_SentimentPlot(df=df, date=varDate, target='trump', channel='tb_twit')
    plotTrumpReddTB = create_SentimentPlot(df=df, date=varDate, target='trump', channel='tb_redd')
    # 3.2. Create Plot (bar): Sentiment Analysis for Biden
    plotBidenTwitVD = create_SentimentPlot(df=df, date=varDate, target='biden', channel='twit')
    plotBidenReddVD = create_SentimentPlot(df=df, date=varDate, target='biden', channel='redd')
    plotBidenTwitTB = create_SentimentPlot(df=df, date=varDate, target='biden', channel='tb_twit')
    plotBidenReddTB = create_SentimentPlot(df=df, date=varDate, target='biden', channel='tb_redd')
    
    varPlot = [plotMention, plotStanceVader, plotStanceTB, plotTrumpTwitVD, plotTrumpReddVD, plotTrumpTwitTB, plotTrumpReddTB, plotBidenTwitVD, plotBidenReddVD, plotBidenTwitTB, plotBidenReddTB]
    
    # 4. LDA HTML files (for Twitter, Reddit)
    twitLDA = url_for('static', filename=varDate.replace('/','')+'twit.html')
    reddLDA = url_for('static', filename=varDate.replace('/','')+'redd.html')
    
    varLDA = [twitLDA, reddLDA]
    
    return render_template('index.html', form=form, varDate=varDate, varLDA=varLDA, varPlot=varPlot)

app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
